In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import random
from numpy import array
import math
import cython
from matplotlib.pyplot import scatter
%matplotlib notebook

In [2]:
def tanh(x, derivative=False):
    if (derivative == True):
        return (1 - (np.tanh(x) ** 2))
    return np.tanh(x)

def relu(x, derivative=False):
    if (derivative == True):
        return 1. * (x > 0)
    return x * (x > 0)

def sigmoid(x, derivative=False):
    sigm = 1. / (1. + np.exp(-x))
    if derivative:
        return sigm * (1. - sigm)
    return sigm
    
def activation_function(x, derivative=False):
    return tanh(x, derivative)

In [3]:
class NeuralNetwork:
    
    def __init__(self, neuron_list, learning_rate, shrink_rate):
        self.layer_count = len(neuron_list)
        self.neuron_list = neuron_list
        self.biases = [np.random.randn(y, 1) / shrink_rate for y in neuron_list[1:]]
        self.weights = [abs(np.random.randn(y, x)) / shrink_rate for x, y in zip(neuron_list[:-1], neuron_list[1:])]
        self.activations = [np.zeros((x,1)) for x in neuron_list]
        self.deltas = [np.zeros((x,1)) for x in neuron_list]
        self.learning_rate = learning_rate
        
    def forward_propogation(self, x):
        self.activations[0] = x      
        for i in range(self.layer_count-1):
            self.activations[i+1] = activation_function(np.dot(self.weights[i], self.activations[i])+self.biases[i])
        return self.activations[-1]
    
    def compute_deltas(self,output_labels):
        # Compute last layers' activations
        for i in range(self.neuron_list[-1]):
            self.deltas[-1][i] = 2*activation_function(self.activations[-1][i],True)*(output_labels[i]-activation_function(self.activations[-1][i]))            
        
        # Compute all deltas in all layers
        for l in range(self.layer_count-2, 0, -1):
            np.dot(np.dot(np.transpose(self.weights[l]),self.deltas[l+1]), np.transpose(activation_function(np.dot(self.weights[l-1], self.activations[l-1]), True)))
            
    def back_propogation(self, output_labels):
        # Compute deltas
        self.compute_deltas(output_labels)
        # Update weights 
        for l in range(0,self.layer_count-1):
            self.weights[l] += self.learning_rate * np.dot(self.deltas[l+1], np.transpose(self.activations[l]))

                 
    def train(self, x, y, epoch):
        error = []
        for e in range(epoch):
            pass_error = 0
            print("Epoch: " + str(e))
            for i in range(len(x)):
                estimation = self.forward_propogation(x[i])
                #print("Est: " + str(np.transpose(estimation)))
                #print("Out: " + str(np.transpose(y[i])))
                #print("")
                pass_error += 100 * np.sum( ((estimation - y[i])**2) / (y[i]**2) ) / len(estimation)
                self.back_propogation(y[i])
            error.append(pass_error / len(x))
            print("Error: " + str(pass_error/ len(x)))
        return error          